# Paths

In [1]:
path = '/home/tih_isi_7/G-NeuroDAVIS/'
res_plots = path + 'Results/Plots/'
path_emb = path + 'Data/Embeddings/'
path_tab = path + 'Results/Tables/'

# Importing libraries

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append(path)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras
import ipynbname
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import Datasets as ds
from keras.models import load_model
from utils import DownstreamAnalysis as DA

dataname = ipynbname.name().rsplit("_")[0]
txt = %pwd
methodname = txt.rsplit("/")[-1]

# Load labels and embeddings

In [3]:
y, _ = ds.LoadData(dataname, DType='None', LType=True)
x_embedding = np.array(pd.read_csv(path_emb+dataname+'/'+dataname+'_'+methodname+'.csv', index_col=0, header=0))

# Downstream analysis

### k-NN

In [4]:
neighbour = [5, 15, 25, 35, 45]
Score_knn = []
for n in neighbour:
    Score_knn.append(DA.Knn(x_embedding, y, n))
Score_knn = pd.DataFrame(Score_knn, index=['neighbour '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_knn.to_csv(path_tab+dataname+'_'+methodname+'_knn.csv')
Score_knn

,Accuracy,Precision,Recall,F1-Score
neighbour 5,0.838250,0.838259,0.838250,0.837759
neighbour 15,0.849417,0.849821,0.849417,0.849146
neighbour 25,0.847250,0.847782,0.847250,0.846939
neighbour 35,0.845917,0.846560,0.845917,0.845583
neighbour 45,0.843417,0.844188,0.843417,0.843090


### Random Forest

In [5]:
n_tree = [20,40,60,80,100]
Score_rfc = []
for n in n_tree:
    Score_rfc.append(DA.RFC(x_embedding, y, n))
Score_rfc = pd.DataFrame(Score_rfc, index=['n_tree '+ str(s) for s in n_tree],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_rfc.to_csv(path_tab+dataname+'_'+methodname+'_rfc.csv')
Score_rfc

,Accuracy,Precision,Recall,F1-Score
n_tree 20,0.832583,0.832281,0.832583,0.832184
n_tree 40,0.834000,0.834080,0.834000,0.833805
n_tree 60,0.834750,0.835065,0.834750,0.834558
n_tree 80,0.834000,0.834188,0.834000,0.833798
n_tree 100,0.836417,0.836394,0.836417,0.836116


### k-Means

In [6]:
iteration = 10
Results_km = []

for i in tqdm(range(iteration), desc='Progress'):
    Results_km.append(DA.kmeans(x_embedding, y))
    
Results_km = pd.DataFrame(Results_km,columns = ['ARI', 'FMI','NMI'])
Results_km

Progress: 100%|█████████████████████████████████| 10/10 [00:04<00:00,  2.16it/s]


,ARI,FMI,NMI
0,0.310856,0.426016,0.382652
1,0.313443,0.428103,0.385126
2,0.310949,0.426075,0.382734
3,0.314599,0.428682,0.386083
4,0.313421,0.428088,0.385110
5,0.313390,0.428062,0.385087
6,0.312591,0.427063,0.384211
7,0.315149,0.428838,0.386604
8,0.313479,0.428128,0.385169
9,0.313310,0.427988,0.385014


### Agglomerative 

In [ ]:
idx = ['ARI', 'FMI','NMI']
AglScore = pd.DataFrame(DA.Agglomerative(x_embedding, y), index = idx)    #seurat_clusters, Cluster, celltype.l2
AglScore.T